# Json Functions
```python
#Read json
load("file_path.txt")
#Convert str to json
str_to_json(data)
#Write json to txt
write(data, "file_path.txt")
```

In [20]:
def parse_json(s):
    def parse_value(index):
        c = s[index]
        
        if c == '{':
            obj = {}
            index += 1
            while True:
                while s[index] in ' \n\r\t': index += 1
                if s[index] == '}':
                    index += 1
                    break

                key, index = parse_string(index)
                while s[index] in ' \n\r\t': index += 1
                assert s[index] == ':'
                index += 1
                while s[index] in ' \n\r\t': index += 1
                value, index = parse_value(index)
                obj[key] = value
                while s[index] in ' \n\r\t': index += 1
                if s[index] == ',':
                    index += 1
                elif s[index] == '}':
                    continue
            return obj, index
        
        elif c == '[':
            arr = []
            index += 1
            while True:
                while s[index] in ' \n\r\t': index += 1
                if s[index] == ']':
                    index += 1
                    break
                value, index = parse_value(index)
                arr.append(value)
                while s[index] in ' \n\r\t': index += 1
                if s[index] == ',':
                    index += 1
                elif s[index] == ']':
                    continue
            return arr, index
        
        elif c == '"':
            return parse_string(index)
        
        elif c.isdigit() or c == '-':
            return parse_number(index)
        
        elif s.startswith('true', index):
            return True, index + 4
        
        elif s.startswith('false', index):
            return False, index + 5
        
        elif s.startswith('null', index):
            return None, index + 4
        
        else:
            raise ValueError(f"Unexpected character at {index}: {s[index]}")
    
    def parse_string(index):
        assert s[index] == '"'
        index += 1
        result = ''
        while s[index] != '"':
            if s[index] == '\\':
                index += 1
                if s[index] == '"':
                    result += '"'
                elif s[index] == '\\':
                    result += '\\'
                elif s[index] == 'n':
                    result += '\n'
                elif s[index] == 't':
                    result += '\t'
                else:
                    result += s[index]
            else:
                result += s[index]
            index += 1
        index += 1
        return result, index
    
    def parse_number(index):
        num_str = ''
        while index < len(s) and (s[index].isdigit() or s[index] in '-+.eE'):
            num_str += s[index]
            index += 1
        if '.' in num_str or 'e' in num_str or 'E' in num_str:
            return float(num_str), index
        else:
            return int(num_str), index

    value, idx = parse_value(0)
    return value


def load(filepath):
    with open(filepath, 'r', encoding='cp037') as f:
        text = f.read()
    return parse_json(text)

def str_to_json(s):
    return eval(s)

def write(obj, filepath):
    def write_value(obj, indent_level=0):
        indent = ' ' * indent_level
        
        if isinstance(obj, dict):
            items = []
            for key, value in obj.items():
                key_str = write_string(key)
                value_str = write_value(value, indent_level + 2)
                items.append(f"{indent}  {key_str}: {value_str}")
            return "{\n" + ",\n".join(items) + "\n" + indent + "}"
        
        elif isinstance(obj, list):
            items = []
            for value in obj:
                value_str = write_value(value, indent_level + 2)
                if isinstance(value, dict):
                    items.append(f"{indent}  {value_str}")
                else:
                    items.append(f"{indent}  {value_str}")
            return "[\n" + ",\n".join(items) + "\n" + indent + "]"
        
        elif isinstance(obj, str):
            return write_string(obj)
        
        elif isinstance(obj, bool):
            return "true" if obj else "false"
        
        elif obj is None:
            return "null"
        
        elif isinstance(obj, (int, float)):
            return str(obj)
        
        else:
            raise ValueError(f"Unsupported data type: {type(obj)}")

    def write_string(s):
        result = '"'
        for char in s:
            if char == '"':
                result += '\\"'
            elif char == '\\':
                result += '\\\\'
            elif char == '\n':
                result += '\\n'
            elif char == '\t':
                result += '\\t'
            else:
                result += char
        result += '"'
        return result
    
    json_str = write_value(obj)

    #"encrypted" using cp037
    with open(filepath, 'w', encoding='cp037') as f:
        f.write(json_str)

# Data Reader

In [21]:
class datasets:
    def __init__(self):
        self.users = load('datasets/users.txt')
        self.menu = load('datasets/menu.txt')
        self.ingredients = load('datasets/ingredients.txt')
        self.orders = load('datasets/orders.txt')
        self.finance = load('datasets/finance.txt')
        self.feedbacks = load('datasets/feedbacks.txt')

    def get_users(self):
        return self.users

    def get_menu(self):
        return self.menu

    def get_iventory(self):
        return self.ingredients

    def get_orders(self):
        return self.orders

    def get_finance(self):
        return self.finance

    def get_feedback(self):
        return self.feedbacks

# Main system
```python
#Example of data getter
system = system_manager()

for SubClass in (Manager, Cashier, Chef, Customer):
    inst = SubClass()
    decrypted_data = inst.retrieve_data(system)
    decrypted_data = str_to_json(decrypted_data)
    print(f"{SubClass.__name__} decrypted:", decrypted_data)
```

In [22]:
if isinstance(__builtins__, dict):
    _B_CHR = __builtins__['chr']
    _B_ORD = __builtins__['ord']
else:
    _B_CHR = __builtins__.chr
    _B_ORD = __builtins__.ord

In [23]:
class system_manager:
    def __init__(self):
        data_banker = datasets()
        #已认证人员
        self.users_available = data_banker.get_users()

        """加/解密系统"""
        #根据不同role分配不同datasets
        self._plain_data = {
            Manager: {"user": data_banker.get_users(),
                      "order": data_banker.get_orders(),
                      "inventory": data_banker.get_iventory(),
                      "finance": data_banker.get_finance(),
                      "feedback": data_banker.get_feedback()},
            Cashier: {"menu": data_banker.get_menu(),
                      "order": data_banker.get_orders()},
            Chef: {"ingr": data_banker.get_iventory()},
            Customer: {"menu": data_banker.get_menu(),
                      "feedback": data_banker.get_feedback()}
        }
        
        # 密钥
        self.caesar_shift = 3
        self.xor_key = 42
        self.mul_key = 2

    #根据 requester 类型：取出对应明文并加密后返回
    def get_data(self, requester) -> str:
        cls = type(requester)
        if cls not in self._plain_data:
            raise ValueError(f"Unknown requester: {cls}")
        plain = str(self._plain_data[cls])

        if cls is Manager:
            dt = self._encrypt_atbash(plain)
            return dt
        elif cls is Cashier:
            dt = self._encrypt_caesar(plain)
            return dt
        elif cls is Chef:
            dt = self._encrypt_xor(plain)
            return dt
        elif cls is Customer:
            dt = self._encrypt_mul(plain)
            return dt

    # ——四种加密算法——
    def _encrypt_atbash(self, data: str) -> str:
        result = []
        for c in data:
            if 'a' <= c <= 'z':
                result.append(_B_CHR(_B_ORD('z') - (_B_ORD(c) - _B_ORD('a'))))
            elif 'A' <= c <= 'Z':
                result.append(_B_CHR(_B_ORD('Z') - (_B_ORD(c) - _B_ORD('A'))))
            elif '0' <= c <= '9':
                result.append(_B_CHR(_B_ORD('9') - (_B_ORD(c) - _B_ORD('0'))))
            else:
                result.append(c)
        return ''.join(result)

    def _encrypt_caesar(self, data: str) -> str:
        shift = self.caesar_shift
        result = []
        for c in data:
            if 'a' <= c <= 'z' or 'A' <= c <= 'Z':
                base = 'a' if c.islower() else 'A'
                result.append(_B_CHR(
                    (_B_ORD(c) - _B_ORD(base) + shift) % 26
                    + _B_ORD(base)
                ))
            else:
                result.append(c)
        return ''.join(result)

    def _encrypt_xor(self, data: str) -> str:
        key = self.xor_key
        return "".join(f"{(_B_ORD(c) ^ key):02x}" for c in data)

    def _encrypt_mul(self, data: str) -> str:
        key = self.mul_key
        return ",".join(str(_B_ORD(c) * key) for c in data)

    """人员调动系统"""
    def add_user(self, user):
        self.users_available['users'].append(user)
        write(self.users_available, "datasets/users.txt")


# Manager

In [24]:
class Manager:
    """使用阿特巴什（Atbash）加解密——对字母和数字做对称映射"""
    def retrieve_data(self, system) -> str:
        encrypted = system.get_data(self)
        return self.decrypt(encrypted)

    def decrypt(self, data: str) -> str:
        # Atbash 是自身可逆的，加密和解密方法相同
        result = []
        for c in data:
            if 'a' <= c <= 'z':
                # 'a' ↔ 'z', 'b' ↔ 'y' …
                result.append(_B_CHR(_B_ORD('z') - (_B_ORD(c) - _B_ORD('a'))))
            elif 'A' <= c <= 'Z':
                result.append(_B_CHR(_B_ORD('Z') - (_B_ORD(c) - _B_ORD('A'))))
            elif '0' <= c <= '9':
                result.append(_B_CHR(_B_ORD('9') - (_B_ORD(c) - _B_ORD('0'))))
            else:
                result.append(c)
        return ''.join(result)

    def get_item(self):
        system = system_manager()
        decrypted_data = self.retrieve_data(system)
        decrypted_data = str_to_json(decrypted_data)
        return decrypted_data

    def system_administration(self):
        data = self.get_item()
        data = data['users']
        return data

    def approve(self, panel, user_name):
        # 寻找待审批用户并批准
        self.main_system = system_manager
        for user in panel.pending_users:
            if user.name == user_name:
                panel.pending_users.remove(user)  # 从待审批列表中移除
                self.main_system.add_user(user)  # 将用户添加到主系统中
                print(f"管理员批准了用户 {user_name} 的注册。")
                return
        print(f"未找到名为 {user_name} 的待审批用户。")

    def reject(self, panel, user_name):
        # 拒绝注册
        for user in panel.pending_users:
            if user.name == user_name:
                panel.pending_users.remove(user)
                print(f"管理员拒绝了用户 {user_name} 的注册。")
                return
        print(f"未找到名为 {user_name} 的待审批用户。")

```python
    def order_management():

    def financial_management():

    def inventory_control():

    def customer_feedback():
```

# Cashier

In [25]:
class Cashier:
    """使用凯撒（Caesar）加解密"""
    def retrieve_data(self, system) -> str:
        encrypted = system.get_data(self)
        return self.decrypt(encrypted, system.caesar_shift)

    def decrypt(self, data: str, shift: int) -> str:
        result = []
        for c in data:
            if 'a' <= c <= 'z' or 'A' <= c <= 'Z':
                base = 'a' if c.islower() else 'A'
                result.append(_B_CHR(
                    (_B_ORD(c) - _B_ORD(base) - shift) % 26
                    + _B_ORD(base)
                ))
            else:
                result.append(c)
        return ''.join(result)

# Chef

In [26]:
class Chef:
    """使用简单的异或（XOR）加解密"""
    def retrieve_data(self, system) -> str:
        encrypted = system.get_data(self)
        return self.decrypt(encrypted, system.xor_key)

    def decrypt(self, data: str, key: int) -> str:
        chars = []
        for i in range(0, len(data), 2):
            byte = int(data[i:i+2], 16) ^ key
            chars.append(_B_CHR(byte))
        return ''.join(chars)

# Customer

In [27]:
class Customer:
    """使用乘法（Mul）加解密——每个字符的码点乘以常数"""
    def retrieve_data(self, system) -> str:
        encrypted = system.get_data(self)
        return self.decrypt(encrypted, system.mul_key)

    def decrypt(self, data: str, key: int) -> str:
        chars = []
        for part in data.split(','):
            num = int(part) // key
            chars.append(_B_CHR(num))
        return ''.join(chars)

# Main Panel

In [11]:
class Panel:
    manager = Manager()
    test = manager.system_administration()
    print(test)

<class '__main__.Manager'>
{'role': 'manager', 'accID': 'Admin', 'pass': 'Admin'}


In [11]:
test['role']

'manager'

In [1]:
class main_system:
    def __init__(self):
        self.users = []  # 存储已注册的用户

    def add_user(self, user):
        self.users.append(user)

    def show_users(self):
        if self.users:
            print("已注册用户：")
            for user in self.users:
                print(user.name)
        else:
            print("没有用户注册。")

class panel:
    def __init__(self):
        self.pending_users = []  # 存储待审批的用户

    def sign_up(self, name):
        user = User(name)  # 创建一个用户实例
        self.pending_users.append(user)  # 将用户添加到待审批列表
        print(f"用户 {name} 已提交注册申请，等待管理员审批。")

    def show_pending(self):
        if self.pending_users:
            print("待审批用户：")
            for user in self.pending_users:
                print(user.name)
        else:
            print("没有待审批的用户。")

class manager:
    def __init__(self, main_system):
        self.main_system = main_system

    def approve(self, panel, user_name):
        # 寻找待审批用户并批准
        for user in panel.pending_users:
            if user.name == user_name:
                panel.pending_users.remove(user)  # 从待审批列表中移除
                self.main_system.add_user(user)  # 将用户添加到主系统中
                print(f"管理员批准了用户 {user_name} 的注册。")
                return
        print(f"未找到名为 {user_name} 的待审批用户。")

    def reject(self, panel, user_name):
        # 拒绝注册
        for user in panel.pending_users:
            if user.name == user_name:
                panel.pending_users.remove(user)
                print(f"管理员拒绝了用户 {user_name} 的注册。")
                return
        print(f"未找到名为 {user_name} 的待审批用户。")

class User:
    def __init__(self, name):
        self.name = name


# 示例使用
system = main_system()  # 创建系统实例
panel_instance = panel()  # 创建注册面板实例
manager_instance = manager(system)  # 创建管理员实例

# 用户注册申请
panel_instance.sign_up("Alice")
panel_instance.sign_up("Bob")

# 显示待审批的用户
panel_instance.show_pending()

# 管理员批准用户
manager_instance.approve(panel_instance, "Alice")

# 显示已注册的用户
system.show_users()

# 显示剩余待审批的用户
panel_instance.show_pending()

# 管理员拒绝用户
manager_instance.reject(panel_instance, "Bob")


用户 Alice 已提交注册申请，等待管理员审批。
用户 Bob 已提交注册申请，等待管理员审批。
待审批用户：
Alice
Bob
管理员批准了用户 Alice 的注册。
已注册用户：
Alice
待审批用户：
Bob
管理员拒绝了用户 Bob 的注册。
